In [1]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

source_dir = '/Users/mdeverna/Documents/Projects/mean_field_quals/src'
sys.path.insert(0,source_dir)


from simulations import run_simulation, get_peak_day, deriv_simple, deriv_with_homophily

In [2]:
S_o, S_m, I_o, I_m, R_o, R_m, r0s = run_simulation(
    frac_ord = .5,
    prop_infec=.001,
    num_days=100,
    beta_ord=.3,
    recovery_days=7,
    beta_mult=2,
    w_homophily=True,
    alpha = .9,
    mixed=False,
    counts=False,
    N=None,
)

In [3]:
# Fraction of ordinary people
x = .5

# All beta values
beta = .3
beta_mult = 3

# Less beta values highlighted for one of the later plots
alphas = [0.5, 0.6, 0.7, 0.8, 0.9, 1.]
alphas = np.arange(.5,1.02,.02)

totals = []      # Total infections
r0s_ = []        # R0 valuyes
peak_days = []   # Date of each peak

# Will store the progression of infections over time, indexed by beta values
infection_flows = dict()

totals_dict = dict()


for mix in [True, False]:
    
    totals = []      # Total infections
    
    for alpha in alphas:

        # Run the simulation based on the input 
        S_o, S_m, I_o, I_m, R_o, R_m, r0s = run_simulation(
            frac_ord=x,
            prop_infec=.001,
            num_days=100,
            beta_ord=beta,
            recovery_days=5,
            beta_mult=beta_mult,     # Multiply the current beta value by this to get the misinfo group beta
            w_homophily=True,
            alpha=alpha,
            mixed=mix,
        )
        total_ord_inf = max(R_o)
        total_mis_inf = max(R_m)

        infection_flows[alpha] = I_o

        r0s_.append( (r0s[0], r0s[1], r0s[2], beta) )

        totals.append((alpha, total_ord_inf, total_mis_inf, total_ord_inf + total_mis_inf))
    
    totals_df = pd.DataFrame(totals, columns = ['alpha', 'ord_inf', 'mis_inf', 'total'])
    totals_df = totals_df.melt(id_vars='alpha')

    var_map ={
        'mis_inf':'misinformed',
        'ord_inf':'ordinary',
        'total' : 'all'
    }

    totals_df.variable = totals_df.variable.map(var_map)
    
    print(mix)
    if mix:
        name = "mixed"
    else:
        name = "not mixed"
    
    totals_dict[name] = totals_df

True
False


In [4]:
len(totals_dict.keys())

2

In [5]:
totals_dict['mixed'][np.round(totals_dict['mixed']['alpha'],2) == .98]

,alpha,variable,value
24,0.98,ordinary,0.300652
50,0.98,misinformed,0.497307
76,0.98,all,0.797958


In [6]:
totals_dict['mixed'][np.round(totals_dict['mixed']['alpha'],2) == .98]

,alpha,variable,value
24,0.98,ordinary,0.300652
50,0.98,misinformed,0.497307
76,0.98,all,0.797958


In [7]:
totals_dict['not mixed'][np.round(totals_dict['not mixed']['alpha'],2) == .98]

,alpha,variable,value
24,0.98,ordinary,0.300087
50,0.98,misinformed,0.497309
76,0.98,all,0.797396


In [8]:
totals_dict['not mixed'][np.round(totals_dict['not mixed']['alpha'],2) == .98]

,alpha,variable,value
24,0.98,ordinary,0.300087
50,0.98,misinformed,0.497309
76,0.98,all,0.797396


In [9]:
mixed__ = totals_dict['mixed'].copy()
not_mixed__ = totals_dict['not mixed'].copy()

In [10]:
mixed__['type'] = 'mixed'
not_mixed__['type'] = 'not mixed'

In [11]:
both_settings = pd.concat([mixed__, not_mixed__])

In [12]:
both_settings.variable.unique()

array(['ordinary', 'misinformed', 'all'], dtype=object)

In [13]:
shape_styles = {
    'ordinary' : 'circle',
    'misinformed' : 'square',
    'all' : "triangle"
}


alt.Chart(
    both_settings[both_settings['type']=='not mixed'],
).mark_point(size = 100, color='black', opacity=.5).encode(
    x = alt.X(
        'alpha:Q',
        axis = alt.Axis(format=".1f"),
        scale = alt.Scale(domain=(.48,1.02))
    ),
    y = alt.Y(
        "value",
        title="proportion of pop. infected",
        scale=alt.Scale(domain=(0,1))
    ),
#     color = alt.Color(
#         "type:N",
#         title='initial setting',
#         scale = alt.Scale(
#             domain = ['mixed', 'not mixed'],
#             range = ['black', 'red']
#         )
#     ),
    shape = alt.Shape(
        "variable:N",
        title="group",
        scale=alt.Scale(
            domain=list(shape_styles.keys()),
            range=list(shape_styles.values()),
        )
    )
).properties(width=700).configure_axis(
    titleFontSize=15,
    labelFontSize=14,
    labelAngle=0
).configure_legend(
    titleFontSize=15,
    labelFontSize=14
).display(
    scale = 2
)

alt.Chart(...)

In [14]:
mixed = alt.Chart(
    totals_dict['mixed'],
    title = 'mixed'
).mark_point().encode(
    x = alt.X(
        'alpha:Q',
        axis = alt.Axis(format=".1f"),
        scale = alt.Scale(domain=(.48,1.02))
    ),
    y = alt.Y(
        "value",
        title="proportion of pop. infected",
        scale=alt.Scale(domain=(0,1))
    ),
    color = alt.Color(
        "variable",
        title='group'
    )
).properties(width=300)

not_mixed = alt.Chart(
    totals_dict['not mixed'],
    title = 'not mixed'
).mark_point().encode(
    x = alt.X(
        'alpha:Q',
        axis = alt.Axis(format=".1f"),
        scale = alt.Scale(domain=(.48,1.02))
    ),
    y = alt.Y(
        "value",
        axis = alt.Axis(
            title=None,
#             labels=False
        ),
        scale=alt.Scale(domain=(0,1))
    ),
    color = alt.Color(
        "variable",
        title='group'
    )
).properties(width=300)

alt.hconcat(mixed, not_mixed).configure_title(
    fontSize=14,
    offset=-5
).configure_axis(
    titleFontSize=16,
    labelFontSize=14,
    labelAngle=0
).configure_legend(
    titleFontSize=15,
    labelFontSize=14
)

alt.HConcatChart(...)

### Infections over time

In [21]:
# Fraction of ordinary people
x = .5

# All beta values
beta = .3

# Less beta values highlighted for one of the later plots
alphas = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0,]


totals = []      # Total infections
r0s_ = []        # R0 valuyes
peak_days = []   # Date of each peak

# Will store the progression of infections over time, indexed by beta values
infection_flows = dict()


for alpha in alphas:

    # Run the simulation based on the input 
    S_o, S_m, I_o, I_m, R_o, R_m, r0s = run_simulation(
        frac_ord=x,
        prop_infec=.001,
        num_days=100,
        beta_ord=beta,
        recovery_days=5,
        beta_mult=3,     # Multiple the current beta value by this to get the misinfo group beta
        w_homophily=True,
        alpha=alpha,
        mixed=True,
    )
    total_ord_inf = max(R_o)
    total_mis_inf = max(R_m)

    infection_flows[alpha] = I_o

    r0s_.append( (r0s[0], r0s[1], r0s[2], alpha) )

    totals.append((alpha, total_ord_inf, total_mis_inf, total_ord_inf + total_mis_inf))


In [22]:
infections = pd.DataFrame(infection_flows).reset_index()
infections = infections.rename(columns={'index':'day'})
infections.day = infections.day +1
infections = infections.melt(id_vars='day')

infections

,day,variable,value
0,1,0.5,0.000500
1,2,0.5,0.000550
2,3,0.5,0.000650
3,4,0.5,0.000813
4,5,0.5,0.001060
...,...,...,...
595,96,1.0,0.005782
596,97,1.0,0.005370
597,98,1.0,0.004986
598,99,1.0,0.004626


In [23]:
alt.Chart(infections).mark_point(size=50).encode(
    x = alt.X(
        "day:Q",
    ),
    y = alt.Y(
        "value:Q",
        title = "proportion of pop. infected",
#         scale = alt.Scale(domain=(0,.15))
    ),
    color = alt.Color("variable:N", title='alpha'),
    shape = alt.Shape("variable:N")
).properties(
    width=600,
    height=300
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_legend(
    titleFontSize=15,
    labelFontSize=13
)

alt.Chart(...)